# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.7.3 決定木 (Decision Tree, DT)
### 回帰分析

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの分子記述子 (特徴量) で化学構造が数値化されています。特徴量は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を 説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

DT モデルにおいては、説明変数 x の値が計算に用いられることはなく、x の値が何らかの閾値より大きいか小さいかのみが必要です。また目的変数 y の値は平均値の計算のみに用いられます。そこで、一般的に DT モデルを構築するときは、x や y の標準化 (オートスケーリング) は行いません。

DT の実行

In [ ]:
from sklearn.tree import DecisionTreeRegressor # 回帰分析用の DT の実行に使用

In [ ]:
model = DecisionTreeRegressor(max_depth=3, min_samples_leaf=3) # DTモデルの宣言

In [ ]:
model.fit(x_train, y_train) # DT モデル構築

構築された DT モデルの確認

DT モデルは木のような構造で与えられます。その内容を dot ファイル (拡張子が dot であるテキストファイル) に保存します。別途、Graphviz というソフトウェア (無料) を用いて dot ファイルを開けば、DT モデルの内容を可視化できます。Graphviz は [こちら](https://www.graphviz.org/download/) からダウンロードできます。

In [ ]:
from sklearn.tree import export_graphviz # dot ファイル作成に使用

dotファイルの作成。`tree.dot` というファイルが作成されます。

In [ ]:
with open('tree.dot', 'w') as f:
    export_graphviz(model, out_file=f, feature_names=x.columns, class_names=y.name)

`tree.dot` を Graphviz で開くと木の内容を確認できます。

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

クロスバリデーションによる木の深さの最大値の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
max_depths = np.arange(1, 31) # 木の深さの最大値の候補

In [ ]:
max_depths # 念のため確認

クロスバリデーションの設定

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import KFold # クロスバリデーションの分割の設定に使用

In [ ]:
fold = KFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定。(KFold ではなく) StratifiedKFold を使用することで、fold ごとのクラスの割合がなるべく同じになるように分割されます

クロスバリデーションの実行

In [ ]:
from sklearn.model_selection import cross_val_predict # クロスバリデーションに使用

In [ ]:
r2cvs = [] # 空の list。木の深さの最大値の候補ごとに、クロスバリデーション後の r2 を入れていきます

In [ ]:
for max_depth in max_depths:
    model = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=3)
    estimated_y_in_cv = cross_val_predict(model, x_train, y_train, cv=fold)
    r2cvs.append(metrics.r2_score(y_train, estimated_y_in_cv))

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
# 結果の確認
plt.rcParams['font.size'] = 18
plt.scatter(max_depths, r2cvs)
plt.xlabel('maximum depth of tree')
plt.ylabel('r2 in cross-validation')
plt.show()

In [ ]:
optimal_max_depth = max_depths[r2cvs.index(max(r2cvs))] # クロスバリデーション後のr2が最大となる木の深さ

In [ ]:
optimal_max_depth # 念のため確認

DT モデルの構築および予測

In [ ]:
model = DecisionTreeRegressor(max_depth=optimal_max_depth, min_samples_leaf=3) # DTモデルの宣言

In [ ]:
model.fit(x_train, y_train) # DT モデル構築

構築された DT モデルの確認

In [ ]:
from sklearn.tree import export_graphviz # dot ファイル作成に使用

In [ ]:
with open('tree.dot', 'w') as f:
    export_graphviz(model, out_file=f, feature_names=x.columns, class_names=y.name)

`tree.dot` を Graphviz で開くと木の内容を確認できます。

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。